# iTunes Podcasts data preparation

In this code part we will overview datasets and check for the outliers, clean the data. If attributes needs to be standartized or normalized.

We will prepare data for the reporting in Google Studio and for the inferential statistical analysis, too.

In [50]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import os
import itertools
import csv
from datetime import datetime

In [2]:
os.getcwd()

'/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2S2'

## Data loading

In [64]:
input_path = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2S2/Input/database.sqlite"
con = sqlite3.connect(input_path)

podcasts_categories = pd.read_sql("SELECT * FROM categories", con)
podcasts_categories.info()
display(podcasts_categories.head(4))

podcast = pd.read_sql("SELECT * FROM podcasts", con)
podcast.info()
display(podcast.head(4))

reviews = pd.read_sql("SELECT * FROM reviews", con)
reviews.info()
display(reviews.head(4))

podcast_runs = pd.read_sql("SELECT * FROM runs", con)
podcast_runs.info()
display(podcast_runs.head(4))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212372 entries, 0 to 212371
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   podcast_id  212372 non-null  object
 1   category    212372 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


,podcast_id,category
0,c61aa81c9b929a66f0c1db6cbe5d8548,arts
1,c61aa81c9b929a66f0c1db6cbe5d8548,arts-performing-arts
2,c61aa81c9b929a66f0c1db6cbe5d8548,music
3,ad4f2bf69c72b8db75978423c25f379e,arts


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110024 entries, 0 to 110023
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   podcast_id  110024 non-null  object
 1   itunes_id   110024 non-null  int64 
 2   slug        110024 non-null  object
 3   itunes_url  110024 non-null  object
 4   title       110024 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.2+ MB


,podcast_id,itunes_id,slug,itunes_url,title
0,a00018b54eb342567c94dacfb2a3e504,1313466221,scaling-global,https://podcasts.apple.com/us/podcast/scaling-...,Scaling Global
1,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando
2,a0004b1ef445af9dc84dad1e7821b1e3,139076942,mystery-dancing-in-the-dark,https://podcasts.apple.com/us/podcast/mystery-...,Mystery: Dancing in the Dark
3,a00071f9aaae9ac725c3a586701abf4d,1332508972,kts-money-matters,https://podcasts.apple.com/us/podcast/kts-mone...,KTs Money Matters


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2067529 entries, 0 to 2067528
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   podcast_id  object
 1   title       object
 2   content     object
 3   rating      int64 
 4   author_id   object
 5   created_at  object
dtypes: int64(1), object(5)
memory usage: 94.6+ MB


,podcast_id,title,content,rating,author_id,created_at
0,c61aa81c9b929a66f0c1db6cbe5d8548,really interesting!,Thanks for providing these insights. Really e...,5,F7E5A318989779D,2018-04-24T12:05:16-07:00
1,c61aa81c9b929a66f0c1db6cbe5d8548,Must listen for anyone interested in the arts!!!,Super excited to see this podcast grow. So man...,5,F6BF5472689BD12,2018-05-09T18:14:32-07:00
2,ad4f2bf69c72b8db75978423c25f379e,nauseatingly left,"I'm a liberal myself, but its pretty obvious a...",1,1AB95B8E6E1309E,2019-06-11T14:53:39-07:00
3,ad4f2bf69c72b8db75978423c25f379e,Diverse stories,I find Tedx talks very inspirational but I oft...,5,11BB760AA5DEBD1,2018-05-31T13:08:09-07:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   run_at         16 non-null     object
 1   max_rowid      16 non-null     int64 
 2   reviews_added  16 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 512.0+ bytes


,run_at,max_rowid,reviews_added
0,2021-05-10 02:53:00,3266481,1215223
1,2021-06-06 21:34:36,3300773,13139
2,2021-07-02 18:04:55,3329699,11561
3,2021-08-01 17:54:42,3360315,11855


## Data overview & preparation

### Checking podcasts categories values

- What unique categories we have?
- Do we need to make any fix'es for the standartization?

In [6]:
podcasts_categories.category.unique()

array(['arts', 'arts-performing-arts', 'music', 'arts-design',
       'education', 'society-culture', 'arts-visual-arts', 'leisure',
       'leisure-hobbies', 'arts-books', 'technology',
       'society-culture-personal-journals',
       'society-culture-relationships', 'fiction',
       'fiction-science-fiction', 'arts-food',
       'education-language-learning', 'fiction-drama', 'true-crime',
       'health-fitness', 'health-fitness-nutrition', 'history', 'comedy',
       'arts-fashion-beauty', 'tv-film', 'society-culture-places-travel',
       'comedy-comedy-interviews', 'kids-family',
       'health-fitness-mental-health', 'religion-spirituality',
       'business', 'news', 'news-business-news', 'education-courses',
       'society-culture-philosophy', 'leisure-video-games',
       'tv-film-film-reviews', 'leisure-games', 'spirituality',
       'tv-film-film-history', 'business-careers',
       'business-entrepreneurship', 'science', 'science-earth-sciences',
       'comedy-improv'

We see that podcasts categories has duplicates e.g. `tv-film-film-interviews`, `fiction-comedy-fiction`.

Categories might be not ordered. If we will want to aggregate the data and count # of unique categories or podcasts distribution per it it might be some disprepancies and not aggregate in certain categories groups.

Thus we will modify and standartize podcats categories in next steps od data preparation.

### Categories per podcats overview

We gonna check `cateogies` dataset:
- How many unique podcats this dataset has?
- How many categories each podcasts has? 
- Does podcats has multiple categories?

In [65]:
podcats_categories_agg = pd.read_sql_query("""SELECT 
    podcast_id,
    COUNT(*) AS occur,
    COUNT(DISTINCT(category)) AS unique_cat,
    GROUP_CONCAT(category) AS cat_agg
FROM categories
GROUP BY 1
ORDER BY occur DESC, podcast_id
            """, con)
podcats_categories_agg

,podcast_id,occur,unique_cat,cat_agg
0,be905117ff9b3a13b1a065801241903b,8,8,"arts,business,business-entrepreneurship,christ..."
1,c4be70db3c16d3607bd7ae4999cb6e86,8,8,"business,business-careers,business-entrepreneu..."
2,fa8c359031380cd4f3b38358c8a75f92,8,8,"business,business-careers,education,education-..."
3,a7ae747331094b205de1e093011f7e1e,7,7,"business,business-careers,education,health-fit..."
4,af4c9ab7e54007e755ed9f063b89a328,7,7,"business,business-entrepreneurship,christianit..."
...,...,...,...,...
110019,fffe308414050768d3ce3782aa503b7d,1,1,tv-film
110020,ffff19eec34f8f036a2c1e146f79c857,1,1,sports
110021,ffff66f98c1adfc8d0d6c41bb8facfd0,1,1,comedy
110022,ffff923482740bc21a0fe184865ec2e2,1,1,education


In [10]:
podcats_categories_agg.cat_agg[1]

'business,business-careers,business-entrepreneurship,business-investing,news,news-business-news,society-culture,society-culture-personal-journals'

Categories data set has 110 024 unique podcasts (even it has 212 372 row entries).

Some of podcasts have several categories labeled (e.g. `be905117ff9b3a13b1a065801241903b` podcasts has 8 entires and 8 unique categories). Some only one single category (e.g. `ffffbd44ec5f79d502f16ae372bf2d4f`)

Some podcasts might have repetitive key words of categories, e.g. `business,business-careers,business-entrepreneurship,business-investing,news,news-business-news,society-culture,society-culture-personal-journals`. Thus we will standartize podcasts categories.

### Podcasts categories standartization

In [66]:
def unique_category(txt):
    """This function extracts unique categories values:
    1. We split string values to list
    2. We sort list values alphabetically
    3. We select only distint categories values
    4. Finaly join categories to single string
    After these steps we return modified and prepared for analysis unique categories"""
    return '-'.join(np.unique(sorted(txt.replace(',','-').split('-'))))


def count_categories(txt):
    """This function counts how many categories podcats has labeled"""
    return len(txt.split('-'))

# Example
print("Before modification:", podcats_categories_agg.cat_agg[1])
print("After modification:", unique_category(podcats_categories_agg.cat_agg[1]))

podcats_categories_agg['category_fixed'] = (podcats_categories_agg.cat_agg
                                            .apply(lambda x: unique_category(x)))
podcats_categories_agg['CategoriesNo'] = (podcats_categories_agg.category_fixed
                                            .apply(lambda x: count_categories(x)))

podcats_categories_agg = podcats_categories_agg.drop(columns=['occur','unique_cat'])
podcats_categories_agg.head(5)

Before modification: business,business-careers,business-entrepreneurship,business-investing,news,news-business-news,society-culture,society-culture-personal-journals
After modification: business-careers-culture-entrepreneurship-investing-journals-news-personal-society


,podcast_id,cat_agg,category_fixed,CategoriesNo
0,be905117ff9b3a13b1a065801241903b,"arts,business,business-entrepreneurship,christ...",arts-business-christianity-culture-education-e...,11
1,c4be70db3c16d3607bd7ae4999cb6e86,"business,business-careers,business-entrepreneu...",business-careers-culture-entrepreneurship-inve...,9
2,fa8c359031380cd4f3b38358c8a75f92,"business,business-careers,education,education-...",business-careers-culture-education-improvement...,12
3,a7ae747331094b205de1e093011f7e1e,"business,business-careers,education,health-fit...",business-careers-culture-education-fitness-hea...,9
4,af4c9ab7e54007e755ed9f063b89a328,"business,business-entrepreneurship,christianit...",business-christianity-education-entrepreneursh...,9


`category_fixed` - column is standartized with unique key words of categories representing what podcats is about.

### Checking duplicates of podcasts

We will check if `podcasts` dataset contains only unique podcasts and wheter any duplicate values appears.

In [14]:
pd.read_sql_query("""SELECT 
    COUNT(*) AS occur,
    COUNT(DISTINCT(podcast_id)) AS unique_podcasts_ids,
    COUNT(DISTINCT(itunes_id)) AS unique_itunes_ids
FROM podcasts
            """, con)

,occur,unique_podcasts_ids,unique_itunes_ids
0,110024,110024,110024


Number of rows, number of unique podcasts ids and itunes ids is the same. Thus we can assume, that `podcasts` dataset contains only unique podcats values. There is no duplicates.

### Podcats & categories mapping

We want to check if all podcasts has categories and if all podcats in `categories` dataset has entry in `podcasts` dataset.

In [20]:
check1 = pd.read_sql_query("""SELECT 
    DISTINCT p.podcast_id
FROM podcasts AS p
LEFT JOIN categories c
    ON p.podcast_id = c.podcast_id
WHERE
    c.podcast_id IS NULL
    OR c.category IS NULL OR c.category = ''
""", con)
check1

,podcast_id


All podcasts in `podcasts` dataset mapped with `categories` dataset podcasts and not missing categories values were found.

In [22]:
check2 = pd.read_sql_query("""SELECT 
    DISTINCT p.podcast_id
FROM categories c
LEFT JOIN podcasts AS p
    ON p.podcast_id = c.podcast_id
WHERE
    p.podcast_id IS NULL
""", con)
check2

,podcast_id


No not mapped podcats were found from `categories` dataset, too.

### Reviews overview

We will check `reviews` dataset:
- How many reviews were collected?
- How many unique podcasts got reviews?
- How many unique listeners left reviews?
- The time range reviews were collected.

In [25]:
pd.read_sql_query("""SELECT 
    COUNT(*) AS occur,
    COUNT(DISTINCT(podcast_id)) AS unique_values,
    COUNT(DISTINCT(author_id)) AS unique_authors,
    MIN(created_at) AS first_review_time,
    MAX(created_at) AS last_review_time
FROM reviews
""", con)

,occur,unique_values,unique_authors,first_review_time,last_review_time
0,2067529,111544,1475285,2005-12-09T21:41:01-07:00,٢٠٢٠-١٢-٢٤T١١:٥٥:٠٨-07:00


In total 2 067 529 reviews were collected from 1 475 285 unique listeners.
We can see that 111 544 unique podcasts ids were found in this dataset, but from `podcasts` and `categories` we saw less unique podcasts. Why does it differ?

Besides, we can see that last_review_time is has strange value, not date format. We will investigate and fix it in next steps

- Is where any podcasts which got review but was not listed in podcasts dataset?

In [30]:
check3 = pd.read_sql_query("""SELECT 
    DISTINCT r.podcast_id
FROM reviews r
LEFT JOIN podcasts AS p
    ON r.podcast_id = p.podcast_id
WHERE
    p.podcast_id IS NULL
""", con)
print('Number of missing podcasts in reviews dataset compared to podcasts dataset:',len(check3))
check3.head(3)

Number of missing podcasts in reviews dataset compared to podcasts dataset: 1520


,podcast_id
0,f2ff762ba0763f124b5c64d728888a30
1,cdb26d8a1ce63f29c854d28557495054
2,a7d10181051177d55b69a307fa80e0d7


There is 1520 podcasts having reviews but not listed in the `podcasts` dataset.

- Is there any podcast which did not get review?

In [32]:
check4 = pd.read_sql_query("""SELECT 
    DISTINCT r.podcast_id
FROM podcasts AS p 
LEFT JOIN reviews r
    ON r.podcast_id = p.podcast_id
WHERE
    p.podcast_id IS NULL
""", con)
print('Number of missing podcasts in podcasts  dataset compared to reviews dataset:',len(check4))
check4.head(3)

Number of missing podcasts in podcasts  dataset compared to reviews dataset: 0


,podcast_id


It looks like all podcasts in `podcasts` dataset got at least one review. There was no such podcasts without a review.

- What is the unique reviews ratings values?

In [36]:
reviews.rating.unique()

array([5, 1, 3, 4, 2])

All reviews ratings are numeric values from 1 to 5.

Let's eliminate strange reviews dates from reviews dataset

In [70]:
reviews = reviews[(reviews.created_at != '٢٠٢٠-١٢-٢٤T١١:٥٥:٠٨-07:00')
                  &
                  (reviews.created_at != '٢٠٢٠-١٢-٢٤T٠٨:٢٦:٣٦-07:00')
                 ].reset_index()

reviews['ReviewDate'] = (reviews.created_at
                             .apply(lambda x: datetime.strptime(x[0:10],'%Y-%m-%d')))

print('First review data:',reviews.ReviewDate.min())
print('Last review data: ',reviews.ReviewDate.max())

First review data: 2005-12-09 00:00:00
Last review data:  2023-02-16 00:00:00


The reviews were collected from 2005-12-09 to 2023-02-16

- What is the avearage rating per podcats?
- How many reviews they got

In [108]:
podcast_reviews_agg = pd.read_sql_query("""SELECT 
    podcast_id,
    COUNT(*) AS num_of_reviews,
    AVG(rating) AS avg_rating
FROM reviews
GROUP BY podcast_id
ORDER BY num_of_reviews
""", con)
podcast_reviews_agg

,podcast_id,num_of_reviews,avg_rating
0,a00018b54eb342567c94dacfb2a3e504,1,5.000000
1,a00043d34e734b09246d17dc5d56f63c,1,5.000000
2,a0004b1ef445af9dc84dad1e7821b1e3,1,1.000000
3,a000b6538cca860a11a94607c13e9595,1,5.000000
4,a00173e86c410b244d2e2a4fa7f4b658,1,5.000000
...,...,...,...
111539,f2377a9b0d9a2e0fb05c3dad55759328,7389,4.735553
111540,f5fce0325ac6a4bf5e191d6608b95797,8248,3.801285
111541,bad6c91efdbee814db985c7a65199604,9698,4.776243
111542,bc5ddad3898e0973eb541577d1df8004,10675,3.605714


## Data exports

### Data export for the report about podcasts

In [79]:
podcast_data = pd.merge(podcast, podcasts_categories, 
                        how='left', 
                        on='podcast_id')
podcast_df = pd.merge(podcast_data, podcats_categories_agg, 
                      how='left', 
                      on='podcast_id')
podcast_df.head(5)

,podcast_id,itunes_id,slug,itunes_url,title,category,cat_agg,category_fixed,CategoriesNo
0,a00018b54eb342567c94dacfb2a3e504,1313466221,scaling-global,https://podcasts.apple.com/us/podcast/scaling-...,Scaling Global,business,business,business,1
1,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando,christianity,"christianity,religion-spirituality",christianity-religion-spirituality,3
2,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando,religion-spirituality,"christianity,religion-spirituality",christianity-religion-spirituality,3
3,a0004b1ef445af9dc84dad1e7821b1e3,139076942,mystery-dancing-in-the-dark,https://podcasts.apple.com/us/podcast/mystery-...,Mystery: Dancing in the Dark,religion-spirituality,"religion-spirituality,spirituality",religion-spirituality,2
4,a0004b1ef445af9dc84dad1e7821b1e3,139076942,mystery-dancing-in-the-dark,https://podcasts.apple.com/us/podcast/mystery-...,Mystery: Dancing in the Dark,spirituality,"religion-spirituality,spirituality",religion-spirituality,2


In [116]:
output_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2S2/Output/podcast_data.csv"
podcast_df[['podcast_id','itunes_id','category_fixed','CategoriesNo',
           'num_of_reviews','avg_rating']].to_csv(output_csv, index=False)

### Data export for the report about reviews traffic

In [83]:
reviews_data_agg = (reviews.groupby('ReviewDate')
                           .agg(NumOfReviews=('podcast_id','count'))
                           .reset_index())
reviews_data_agg.head(3)     

,ReviewDate,NumOfReviews
0,2005-12-09,3
1,2005-12-10,9
2,2005-12-11,5


In [84]:
output_reviews_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2S2/Output/reviews_data.csv"
reviews_data_agg.to_csv(output_reviews_csv, index=False)

### Data export for the inferential analysis

In [93]:
podcats_reviews = pd.merge(podcast_df, reviews, how='inner', on = 'podcast_id')
podcats_reviews = podcats_reviews.drop(columns=['level_0','index'])
podcats_reviews = podcats_reviews.rename(columns={'title_x':'podcast_title',
                                                 'title_y':'review_title'})
podcats_reviews.info()
podcats_reviews.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4529481 entries, 0 to 4529480
Data columns (total 15 columns):
 #   Column          Dtype         
---  ------          -----         
 0   podcast_id      object        
 1   itunes_id       int64         
 2   slug            object        
 3   itunes_url      object        
 4   podcast_title   object        
 5   category        object        
 6   cat_agg         object        
 7   category_fixed  object        
 8   CategoriesNo    int64         
 9   review_title    object        
 10  content         object        
 11  rating          int64         
 12  author_id       object        
 13  created_at      object        
 14  ReviewDate      datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(11)
memory usage: 552.9+ MB


,podcast_id,itunes_id,slug,itunes_url,podcast_title,category,cat_agg,category_fixed,CategoriesNo,review_title,content,rating,author_id,created_at,ReviewDate
0,a00018b54eb342567c94dacfb2a3e504,1313466221,scaling-global,https://podcasts.apple.com/us/podcast/scaling-...,Scaling Global,business,business,business,1,Very informative,Great variety of speakers!,5,CC47C85896D423B,2017-11-29T12:16:43-07:00,2017-11-29
1,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando,christianity,"christianity,religion-spirituality",christianity-religion-spirituality,3,Good Sernons,I'm a regular listener. I only wish that the ...,5,103CC9DA2046218,2019-10-08T04:23:32-07:00,2019-10-08
2,a00043d34e734b09246d17dc5d56f63c,158973461,cornerstone-baptist-church-of-orlando,https://podcasts.apple.com/us/podcast/cornerst...,Cornerstone Baptist Church of Orlando,religion-spirituality,"christianity,religion-spirituality",christianity-religion-spirituality,3,Good Sernons,I'm a regular listener. I only wish that the ...,5,103CC9DA2046218,2019-10-08T04:23:32-07:00,2019-10-08


In [94]:
output_reviews_csv = "/Users/kristinazekiene/Documents/LearningPython/TuringCollege/M2S2/Output/podcasts_reviews_data.csv"
podcats_reviews.to_csv(output_reviews_csv, index=False)